### Beispiel eines Neuronalen Netzes zur Vorhersage von Kundenabwanderung
Genutzt wird in diesem Fall ein **LSTM**-Netz (Long-Short-Term-Memory)
, welches besonders gut für die Analyse von längeren Sequenzen (Texte, Kursverläufe etc.) geeignet ist.
In unserem Fall sind dies Bestellverläufe verschiedener Kunden. Ziel ist es, das Netz auf einen Teil der Kundenbestellungen zu trainieren, und mit dem Rest zu testen ob das System diese richtig als abgesprungen oder nicht abgesprungen klassifiziert.

Im folgenden finden Sie die Codefragmente mit den zugehörigen Beschreibungen. Diese müssen nacheinander von Ihnen ausgeführt werden.

Import der notwendigen Bibliotheken:
* **Keras** - Hauptframework für Machine Learning. (Basiert auf Google's Tensorflow)
* **Numpy** - Unter anderem zur Durchführung von Vektorberechnungen.
* **Pandas** - Regelt den Import und die Formatierung von Daten. 

In [ ]:
import random as rn
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from dateutil.relativedelta import relativedelta
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Masking, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras_preprocessing.sequence import pad_sequences
from numpy.random import seed

Funktion, welche einen *Inner-Join* auf zwei Datensätzen ausführt.

In [ ]:
def drop_missing_values_vise_versa(df1, df2, column_name):
    drop_condition = df1[column_name].isin(df2[column_name]) == False
    df1 = df1.drop(df1[drop_condition].index)
    drop_condition = df2[column_name].isin(df1[column_name]) == False
    df2 = df2.drop(df2[drop_condition].index)
    return df1, df2

Funktion, welche fehlende Daten einer Zeitreihe auffüllt.

In [ ]:
def fill_dates(df, end_date):
    start_date = df['date'].iloc[0]
    if start_date > end_date or len(df) == 0:
        return df
    diff = ((end_date.year - start_date.year) * 12 + end_date.month - start_date.month)
    date_list = [start_date + relativedelta(months=x) for x in range(0, diff)]
    date_frame = pd.DataFrame(date_list)
    date_frame.columns = ['date']
    value = pd.merge(date_frame, df, how='left').fillna(0)
    value = value.drop(['date'], axis=1)
    return value

Funktion, welche einen Datensatz in einem gewählten Verhältnis teilt.

In [ ]:
def train_test_split(df, ratio):
    train = np.array(df[:round(len(df) * ratio)])
    test = np.array(df[-round(len(df) * (1 - ratio)):])
    return train, test

Funktion, welche *Random-Seeds* setzt.

In [ ]:
def set_random_seed(s):
    seed(s)
    rn.seed(s)
    tf.random.set_seed(s)

Einlesen der Daten mithilfe von **Pandas**
* Es wird zwischen X und Y-Werten unterschieden.
* Die X-Werte sind in unserem Fall die Bestellverläufe.
* Der Y-Wert ist ein *boolean*-Wert, welcher besagt ob ein Kunde abgesprungen ist.

In [ ]:
x_data = pd.read_csv("cleaned_orders.csv", header=0, index_col=0, sep=",", decimal=".", dtype={0:int})
x_data['date'] = pd.to_datetime(x_data.date, format='%Y-%m-%d')
x_data = x_data.drop("quantity", axis=1)
x_data = x_data[x_data.date < '2018-09-01']


In [ ]:
y_data = pd.read_csv("cleaned_y_data.csv", header=0, index_col=0, sep=",", dtype={0:int,1:int})
y_data = y_data.sort_values('recipient').reset_index(drop=True)

In [ ]:
x_data, y_data = drop_missing_values_vise_versa(x_data, y_data, 'recipient')

Kleiner Ausschnitt der X und Y-Werte:

In [ ]:
x_data.head(10)

In [ ]:
y_data.head(10)

Die drei folgenden Code-Zellen dienen der Transformation der X-Werte in ein 3D-Array, um diese für das Netzwerk lesbar zu machen.

In [ ]:
x_data_dict =  dict()
x_data_grouped = x_data.groupby('recipient')
for recipient in x_data_grouped.groups:
    x_data_dict[recipient] = pd.DataFrame(x_data_grouped['date','sales'].get_group(recipient))
assert len(x_data_dict) == len(y_data)

In [ ]:
X = list()
for key, item in x_data_dict.items():
    X.append(fill_dates(item, datetime.strptime("2018-09-01","%Y-%m-%d")).values.tolist())

In [ ]:
X = pad_sequences(X, value=-1,dtype='float32')
y = np.array(y_data['churned'].values)
y = y.reshape(y.shape[0],1)

Aufteilung der Daten in 70% Trainings- und 30% Testdaten:

In [ ]:
X_test, X_train = train_test_split(X, 0.3)
y_test, y_train = train_test_split(y, 0.3)

**An dieser Stelle sind Sie gefragt..**
Das nachfolgende Codefragment enthält 4 modifizierbare Parameter, welche essenziell für die Genauigkeit des Netzwerks sind.
Diese sind so initialiert, dass das Netwerk ohne Veränderung eine schlechte Performance aufweist.
Ihre Aufgabe ist es nun diese Parameter so anzupassen, dass eine möglichst hohe Genauigkeit erzielt wird.

Die Parameter sind wie folgt beschrieben:
* **learning_rate**: Lernrate des Minimierungs-Algorithmus der Kostenfunktion (default: 0.00001, range: 0-∞)
* **lstm_cells**: Anzahl der Memory-Cells des Netzwerks. Besitzt ein Netzwerk mehr Zellen, kann es sich eine größere Anzahl an Merkmalen merken. Zu viele können zu *Overfitting* führen. (default: 1, range: 1-∞)
* **dropout**: Anteil der Merkmale, die zufällig gelöscht werden. Wirkt *Ovefitting* entgegen (default: 0.5, range: 0-1)
* **epochs**: Anzahl der Traingsdurchläufe (default: 10, range: 1-∞)

In [ ]:
learning_rate = 0.00001
lstm_cells = 1
dropout = 0.5
epochs = 10

In [ ]:
set_random_seed(120)
opt = Adam(learning_rate=learning_rate)
model = Sequential()
model.add(Masking(mask_value=-1, input_shape=(X_train.shape[1],1)))
model.add(LSTM(lstm_cells, recurrent_dropout=dropout))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

Die nachfolgende Code-Zelle enthält den Aufruf der *fit*-Methode, welche das Training des Netzes mit den oben angegebenen Parametern startet. Anschließend wird eine Zusammenfassung des Traingsverlaufs ausgegeben.

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, restore_best_weights=True, patience=5)
history = model.fit(x=X_train,y=y_train,epochs=epochs, verbose=1, callbacks=[es], validation_data=(X_test, y_test))
scores = model.evaluate(X_test, y_test, verbose=0)

plt.rcParams["figure.figsize"] = (16, 5)
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(history.history['loss'], label='Training')
ax1.plot(history.history['val_loss'], label='Testing')
ax1.legend(loc="upper left")
ax1.set_title("Training und Testing Kostenverlauf")
ax2.plot(history.history['acc'], label='Training')
ax2.plot(history.history['val_acc'], label='Testing')
ax2.legend(loc="upper left")#
ax2.set_title("Training und Testing Genauigkeit")
plt.show()

print("Genauigkeit: %.2f%%" % (scores[1]*100))
print(str(round(X_test.shape[0] * scores[1])) + " von " + str(X_test.shape[0]) + " Beispielen richtig klassifiziert!")